- Leave-One-Out分割
- user * item行列の作成
- cos類似度を計算
- 各userに対して，Top-N推薦を生成する
- 正解データの作成
- 評価指標の計算

In [8]:
import numpy as np, pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

print("ok")

ok


In [2]:
cols = ['user_id', 'item_id', 'rating', 'timestamp']

# sep='\t'はタブ区切り
# header=Noneはヘッダ行がないことを示す
# names=colsは列名を指定する
# engine='python'はエンジンをPythonに指定する
df_data = pd.read_csv('u.data', sep='\t', header=None, names=cols, engine='python')

df_data.head(10)

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596
5,298,474,4,884182806
6,115,265,2,881171488
7,253,465,5,891628467
8,305,451,3,886324817
9,6,86,3,883603013


In [5]:
# Leave-One-Out 分割（最新の1件をテストに）
df_data['rank'] = df_data.groupby('user_id')['timestamp'].rank(method='first', ascending=False)
train_df = df_data[df_data['rank'] > 1]
test_df = df_data[df_data['rank'] == 1]

In [6]:
# 行：ユーザ, 列：アイテム、値：評価
user_item_matrix = train_df.pivot(index='user_id', columns='item_id', values='rating').fillna(0)
user_item_matrix.head()

item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,2.0,5.0,5.0,5.0,5.0,5.0,3.0,4.0,5.0,4.0,1.0,4.0,4.0,3.0,4.0,3.0,2.0,4.0,1.0,3.0,3.0,5.0,4.0,2.0,1.0,2.0,2.0,3.0,4.0,3.0,2.0,5.0,4.0,5.0,5.0,4.0,4.0,5.0,3.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,3.0,0.0,0.0,4.0,3.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
# 転置して行：アイテム, 列：ユーザにしてコサイン類似度を計算
item_similarity = cosine_similarity(user_item_matrix.T)
item_similarity = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
print("item_similarity shape: ", item_similarity.shape)
item_similarity.head()

item_similarity shape:  (1680, 1680)


item_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,...,1633,1634,1635,1636,1637,1638,1639,1640,1641,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651,1652,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1663,1664,1665,1666,1667,1668,1669,1670,1671,1672,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.399845,0.327238,0.456747,0.283235,0.117046,0.624408,0.484018,0.496979,0.278088,0.468245,0.465365,0.411722,0.346907,0.569234,0.225473,0.275570,0.046511,0.195048,0.234364,0.338661,0.532087,0.330769,0.470946,0.566463,0.228667,0.209731,0.593609,0.363840,0.153929,0.441598,0.252538,0.290278,0.091201,0.137582,0.109215,0.070227,0.370644,0.280204,0.247609,0.230202,0.372130,0.239762,0.308296,0.250353,0.121290,0.335540,0.320334,0.347747,0.734442,...,0.023734,0.023734,0.023734,0.023734,0.023734,0.023734,0.036152,0.023734,0.023734,0.014826,0.065784,0.010614,0.023734,0.014240,0.023734,0.023734,0.023734,0.023734,0.023734,0.014312,0.0,0.059335,0.047468,0.033055,0.035601,0.052215,0.059335,0.059335,0.035601,0.0,0.0,0.067811,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.033565,0.035601,0.0,0.000000,0.000000,0.035601,0.0,0.0,0.0,0.047468,0.047468
2,0.399845,1.000000,0.276812,0.505551,0.301654,0.083820,0.380486,0.334174,0.253916,0.173166,0.465818,0.459218,0.215413,0.126366,0.249663,0.098487,0.391640,0.113412,0.031125,0.093067,0.321170,0.485424,0.280128,0.430091,0.309427,0.201960,0.409996,0.482208,0.575183,0.090845,0.461834,0.163466,0.474038,0.072565,0.087616,0.215910,0.185160,0.560186,0.330839,0.257458,0.308389,0.369929,0.302256,0.296821,0.170214,0.234648,0.323291,0.305021,0.412239,0.425264,...,0.078540,0.078540,0.078540,0.078540,0.078540,0.078540,0.041612,0.078540,0.078540,0.049064,0.049476,0.035124,0.078540,0.047124,0.078540,0.078540,0.078540,0.078540,0.078540,0.000000,0.0,0.000000,0.078540,0.000000,0.104721,0.000000,0.000000,0.000000,0.104721,0.0,0.0,0.074800,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.055537,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078540,0.078540
3,0.327238,0.276812,1.000000,0.329241,0.216931,0.104482,0.368040,0.202921,0.274181,0.157704,0.365769,0.324206,0.272572,0.191862,0.267942,0.166277,0.408521,0.121173,0.105723,0.190125,0.194728,0.299183,0.237213,0.385007,0.295546,0.216394,0.264071,0.272149,0.255755,0.129416,0.263410,0.189741,0.409220,0.180905,0.046806,0.074109,0.051813,0.243083,0.280387,0.178601,0.232281,0.420814,0.196645,0.225165,0.124135,0.157364,0.265142,0.200567,0.302772,0.343705,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.037003,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.009839,0.0,0.000000,0.097901,0.000000,0.000000,0.000000,0.065268,0.065268,0.097901,0.0,0.0,0.018648,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.032634,0.0,0.0,0.0,0.000000,0.097901
4,0.456747,0.505551,0.329241,1.000000,0.337865,0.090565,0.488451,0.485440,0.422323,0.255575,0.591267,0.587764,0.399831,0.268621,0.333533,0.169721,0.406039,0.101142,0.131155,0.211663,0.257901,0.578965,0.449584,0.415059,0.426038,0.381328,0.371243,0.532890,0.431006,0.220460,0.484879,0.357560,0.468705,0.128297,0.063110,0.171300,0.144261,0.458828,0.458283,0.332197,0.227807,0.473319,0.276922,0.439823,0.253354,0.283677,0.521455,0.416074,0.413376,0.525630,...,0.037716,0.037716,0.037716,0.037716,0.037716,0.037716,0.019982,0.037716,0.037716,0.067737,0.023759,0.016867,0.037716,0.022629,0.037716,0.037716,0.037716,0.037716,0.037716,0.000000,0.0,0.000000,0.056574,0.070036,0.000000,0.000000,0.075431,0.075431,0.000000,0.0,0.0,0.064656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.040004,0.000000,0.0,0.094289,0.094289,0.037716,0.0,0.0,0.0,0.056574,0.075431
5,0.283235,0.301654,0.216931,0.337865,1.000000,0.037596,0.332051,0.250843,0.270515,0.056403,0.373412,0.369870,0.198786,0.104540,0.261332,0.11305

In [11]:
def recommend_items(user_id, top_k=10):
    user_ratings = user_item_matrix.loc[user_id]
    rated_items = user_ratings[user_ratings > 0].index

    scores = pd.Series(dtype=float)

    for item in rated_items:
        sims = item_similarity[item]
        score = sims * user_ratings[item]
        scores = scores.add(score, fill_value=0)

    scores = scores.drop(rated_items, errors='ignore')  # 訓練時に見たアイテムは除外
    return scores.sort_values(ascending=False).head(top_k).index.tolist()

# 全ユーザに対して推薦リスト作成
recommendations = {user: recommend_items(user, top_k=10) for user in test_df['user_id']}

In [13]:
ground_truth = test_df.set_index('user_id')['item_id'].to_dict()
print(ground_truth)

{260: 322, 25: 181, 189: 512, 265: 118, 155: 323, 19: 4, 36: 339, 295: 218, 113: 595, 181: 3, 170: 749, 112: 903, 272: 12, 213: 176, 129: 873, 268: 824, 192: 1061, 282: 325, 201: 272, 146: 1022, 14: 269, 59: 323, 203: 93, 185: 287, 135: 1046, 311: 9, 220: 298, 143: 682, 206: 302, 136: 89, 312: 573, 280: 127, 237: 191, 304: 237, 110: 1246, 214: 1129, 120: 546, 101: 405, 247: 736, 262: 179, 47: 995, 211: 127, 209: 688, 54: 298, 10: 340, 256: 765, 257: 345, 131: 750, 81: 3, 150: 121, 315: 305, 93: 412, 62: 401, 293: 665, 85: 272, 246: 816, 123: 13, 292: 86, 69: 175, 183: 241, 241: 294, 26: 15, 147: 339, 95: 395, 303: 330, 17: 508, 179: 751, 1: 74, 127: 380, 44: 665, 138: 493, 285: 216, 261: 597, 163: 288, 126: 878, 154: 484, 184: 498, 38: 122, 24: 742, 162: 230, 39: 352, 132: 922, 322: 50, 267: 50, 309: 333, 221: 895, 35: 1025, 64: 231, 67: 405, 5: 453, 212: 511, 87: 1189, 169: 174, 102: 1030, 320: 250, 40: 258, 74: 15, 330: 215, 48: 661, 306: 476, 9: 487, 68: 178, 12: 735, 57: 56, 119: 3

In [15]:
from Evaluation_index import recall_at_k, precision_at_k, ndcg_at_k, mrr_at_k, hit_at_k

k = 10
print("=== ItemKNN モデル評価結果（Top-10）===")
print(f"Recall@{k}:    {recall_at_k(recommendations, ground_truth, k):.4f}")
print(f"Precision@{k}: {precision_at_k(recommendations, ground_truth, k):.4f}")
print(f"NDCG@{k}:      {ndcg_at_k(recommendations, ground_truth, k):.4f}")
print(f"MRR@{k}:       {mrr_at_k(recommendations, ground_truth, k):.4f}")
print(f"Hit@{k}:       {hit_at_k(recommendations, ground_truth, k):.4f}")

# RecBoleによるPopularityモデルの評価結果
print("=== RecBole (Popularity Model) 評価結果 ===")
print(f"Recall@10    : {0.0558:.4f}")
print(f"Precision@10 : {0.0549:.4f}")
print(f"NDCG@10      : {0.0716:.4f}")
print(f"MRR@10       : {0.1376:.4f}")
print(f"Hit@10       : {0.3243:.4f}")


=== ItemKNN モデル評価結果（Top-10）===
Recall@10:    0.1177
Precision@10: 0.0118
NDCG@10:      0.0614
MRR@10:       0.0445
Hit@10:       0.1177
=== RecBole (Popularity Model) 評価結果 ===
Recall@10    : 0.0558
Precision@10 : 0.0549
NDCG@10      : 0.0716
MRR@10       : 0.1376
Hit@10       : 0.3243
